In [29]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.patches as mpatches

In [30]:
base = Path("input")
outbreaks_path = base / "Outbreaks.csv"
isocodes_path = base / "isocodes.csv"

In [31]:
# Загружаем и переименовываем нужные колонки
outbreaks = pd.read_csv(outbreaks_path)
outbreaks = outbreaks.rename(columns={
    "Country": "COUNTRY",
    "Year": "YEAR",
    "Disease": "DISEASE"
})

# Фильтруем вспышки, в названии которых упоминается "Ebola"
ebola = outbreaks[outbreaks["DISEASE"].str.contains("Ebola", case=False, na=False)]

# Создаем граф: страны соединяются, если в один и тот же год фиксировалась Эбола
G_e = nx.Graph()
for year, group in ebola.groupby("YEAR"):
    countries = group["COUNTRY"].unique()
    for i, c1 in enumerate(countries):
        for c2 in countries[i+1:]:
            G_e.add_edge(c1, c2, disease="Ebola", year=year)

print("Размер графа:", len(G_e.nodes()), "стран,", len(G_e.edges()), "связей")

# Функция извлекает признаки для пар узлов
def make_features(graph):
    features = []
    labels = []
    pairs = []
    nodes = list(graph.nodes())
    for i, u in enumerate(nodes):
        for v in nodes[i+1:]:
            cn = len(list(nx.common_neighbors(graph, u, v)))
            jaccard = list(nx.jaccard_coefficient(graph, [(u, v)]))[0][2]
            aa = list(nx.adamic_adar_index(graph, [(u, v)]))[0][2]
            pa = list(nx.preferential_attachment(graph, [(u, v)]))[0][2]
            labels.append(1 if graph.has_edge(u, v) else 0)
            features.append([cn, jaccard, aa, pa])
            pairs.append((u, v))
    return np.array(features), np.array(labels), pairs

# Строим признаки и целевую переменную
X_e, y_e, pairs_e = make_features(G_e)

# Разделяем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test, pairs_train, pairs_test = train_test_split(
    X_e, y_e, pairs_e, test_size=0.3, random_state=42
)

# Обучаем логистическую регрессию
model_e = LogisticRegression(max_iter=1000)
model_e.fit(X_train, y_train)

# Вычисляем ROC-AUC на тесте
y_pred_proba_e = model_e.predict_proba(X_test)[:, 1]
auc_e = roc_auc_score(y_test, y_pred_proba_e)
print("ROC-AUC по Эболе:", auc_e)

# Предсказываем вероятности связей для всех пар
y_all_pred_e = model_e.predict_proba(X_e)[:, 1]

# Собираем DataFrame с парами, вероятностями и фактом наличия связи
pred_df_e = pd.DataFrame([
    {"u": u, "v": v, "prob": prob, "real": lab}
    for (u, v), prob, lab in zip(pairs_e, y_all_pred_e, y_e)
])

# Отбираем только те связи, у которых вероятность > 0.5
predicted_edges_e = pred_df_e[pred_df_e["prob"] > 0.5]

# Словарь с переводом названий стран
country_names_ru = {
    "Guinea": "Гвинея",
    "Liberia": "Либерия",
    "Sierra Leone": "Сьерра-Леоне",
    "Uganda": "Уганда",
    "Democratic Republic of the Congo": "ДР Конго",
    "Congo Democratic Republic of the" : "ДР Конго",
    "Congo": "Республика Конго",
    "Côte d'Ivoire": "Кот-д’Ивуар",
    "Mali": "Мали",
    "Philippines": "Филиппины",
    "Senegal": "Сенегал",
    "Spain": "Испания",
    "Tanzania United Republic of": "Танзания",
    "United Kingdom of Great Britain and Northern Ireland": "Великобритания",
    "United States of America": "США",
    "Gabon": "Габон",
    "Nigeria": "Нигерия"
}

# Переименовываем узлы графов
mapping = {node: country_names_ru.get(node, node) for node in G_e.nodes()}
G_e = nx.relabel_nodes(G_e, mapping)

# Создаем предсказанный граф на основе предсказанных связей
G_pred_e = nx.Graph()
G_pred_e.add_edges_from([
    (mapping[u], mapping[v]) for u, v in zip(predicted_edges_e["u"], predicted_edges_e["v"])
])

# Список узлов в алфавитном порядке
nodes_sorted = sorted(G_e.nodes())

# Генерируем цвета для узлов
colors = plt.cm.tab20(np.linspace(0, 1, len(nodes_sorted)))
color_map = {node: colors[i] for i, node in enumerate(nodes_sorted)}

# Вычисляем координаты расположения узлов
pos_e = nx.spring_layout(G_e, seed=42, k=0.3)

# Рисуем графы: реальный и предсказанный
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
nx.draw_networkx_nodes(G_e, pos_e, node_color=[color_map[n] for n in G_e.nodes()], node_size=200)
nx.draw_networkx_edges(G_e, pos_e, alpha=0.6)
plt.title("Реальная сеть Эбола")
plt.axis("off")

plt.subplot(1, 2, 2)
nx.draw_networkx_nodes(G_pred_e, pos_e, node_color=[color_map[n] for n in G_pred_e.nodes()], node_size=200)
nx.draw_networkx_edges(G_pred_e, pos_e, alpha=0.6)
plt.title("Предсказанная сеть Эбола (>0.5)")
plt.axis("off")

plt.tight_layout()
plt.savefig("output/ebola_real_vs_pred_colored.png", dpi=300)
plt.close()

# Рисуем легенду с цветами для стран
plt.figure(figsize=(6, 8))
patches = [mpatches.Patch(color=color_map[node], label=node) for node in nodes_sorted]
plt.legend(handles=patches, loc='center', fontsize=9)
plt.axis("off")
plt.title("Легенда: страны сети Эбола", fontsize=12)
plt.savefig("output/ebola_legend.png", dpi=300)
plt.close()


Размер графа: 16 стран, 55 связей
ROC-AUC по Эболе: 0.9460317460317461
